In [5]:
from obspy.core.event import read_events
import obspy
import numpy as np
import pandas as pd
from scipy.special import sph_harm
import matplotlib.pyplot as plt
import os.path
import shutil
from obspy import UTCDateTime
from netCDF4 import Dataset
from matplotlib import cm
from skimage.filters import gaussian
from mpl_toolkits.basemap import Basemap

dpi = 300
FigFormat = "png"

In [6]:
ExampleInputDir = '../Runs/ExampleSphericalHarmonicsAcousticBall/input'

for imodel in np.arange(0,20):

    ModeName = 'SphericalHarmonicsAcousticBall%04d' %imodel
    EventParDir='../Runs/%s' %(ModeName)  
    if not os.path.exists(EventParDir):
        os.makedirs(EventParDir)
    print(EventParDir, " created")

    if not os.path.exists(EventParDir+'/input'):
        os.makedirs(EventParDir+'/input')

    # copy parameter file
    # shutil.copy(ExampleInputDir+'/TomoNet_LowerMantle__10s.e',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.model.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.nr.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.advanced.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.source.yaml',EventParDir+'/input/')
    # # update event source
    # with open(EventParDir+'/input/inparam.source.yaml','r') as file:
    #     filetxt = file.read()
    # filetxt = filetxt.replace("latitude_longitude: [-56.24, 26.34]", "latitude_longitude: [%.2f, %.2f]" %(EventLat, EventLon))
    # filetxt = filetxt.replace("depth: 46.56e3", "depth: %.1f" %(EventDepth))
    # filetxt = filetxt.replace("data: [2.73e23, 9.08e21, -2.82e23, 6.30e22, 1.90e23, 2.16e23]", "data: [%e, %e, %e, %e, %e, %e]"  %(MomentTensor['m_rr'],MomentTensor['m_tt'],MomentTensor['m_pp'],MomentTensor['m_rt'],MomentTensor['m_rp'],MomentTensor['m_tp']))
    # with open(EventParDir+'/input/inparam.source.yaml','w') as file:
    #     file.write(filetxt)

    shutil.copy(ExampleInputDir+'/inparam.output.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/Synthetic_Stations_Ball.txt',EventParDir+'/input/')

    # # generate random model
    ### Real spherical harmonics
    coeff = {}

    ModelCoeff = dict()
    ModelCoeff['Depth'] = []
    ModelCoeff['l'] = []
    ModelCoeff['m'] = []
    ModelCoeff['Value'] = []

    l_max = 8
    tomorange = 0.1
    DepthList = [0. , 0.2, 0.4, 0.6, 0.8]

    for depth in DepthList:
        coeff[depth] = {}
        
        for l in range(0,l_max+1):

            for m in np.arange(-l,l+1):
                name = '%s_%s' %(l, m)
                # print(l,m)
                ModelCoeff['Depth'].append(depth)
                ModelCoeff['l'].append(l)
                ModelCoeff['m'].append(m)

                Val = (np.random.rand()-0.5)/0.5*tomorange  # random uniform distribution between [0 - 1)
                ModelCoeff['Value'].append(Val)

                coeff[depth][name] = Val       


    df = pd.DataFrame(data=ModelCoeff)
    df.to_pickle(EventParDir+"/Spherical_Harmonics.pkl")

    grid_depth = np.array(DepthList)
    grid_lat = np.linspace(-90, 90, 181)
    grid_lon = np.linspace(-180, 180, 361)

    # Make sure RADISU and Coordinates are ascendingly sorted
    grid_depth.sort()
    grid_lat.sort()
    grid_lon.sort()

    LON, LAT = np.meshgrid(grid_lon, grid_lat)
    MLTomo = np.zeros([len(grid_lat), len(grid_lon), len(grid_depth)])

    SlicingDepth = 0.1
    for i, SlicingDepth in enumerate(DepthList):
        # initiate TomoSum
        TomoSum = np.zeros([len(grid_lat),len(grid_lon)])

        for l in range(0,l_max+1):
            for m in np.arange(-l,l+1):
                # print('l, m = ', l, m)
                name = '%s_%s' %(l, m)
                Y_grid = sph_harm(m, l, np.radians(LON-180), np.radians(90-LAT))

                if m < 0:
                    Y_grid = np.sqrt(2) * (-1)**(-m) * Y_grid.imag
                elif m > 0:
                    Y_grid = np.sqrt(2) * (-1)**m * Y_grid.real

                TomoSum[:,:] = TomoSum[:,:] + coeff[SlicingDepth][name] * Y_grid
        MLTomo[:,:,i] = TomoSum[:,:]
    
    print(MLTomo.min(), MLTomo.max())

    NCName = "degree8_random.nc"
    # write to file
    if os.path.exists(EventParDir+'/input/'+NCName):
        os.remove(EventParDir+'/input/'+NCName)

    nc = Dataset(EventParDir+'/input/'+NCName, 'w')
    nc.createDimension('nlat', size=len(grid_lat))
    nc.createDimension('nlon', size=len(grid_lon))
    nc.createDimension('ndepth', size=len(grid_depth))
    nc.createVariable('latitude', float, dimensions=('nlat'))
    nc['latitude'][:] = grid_lat
    nc.createVariable('longitude', float, dimensions=('nlon'))
    nc['longitude'][:] = grid_lon
    nc.createVariable('depth', float, dimensions=('ndepth'))
    nc['depth'][:] = grid_depth
    nc.createVariable('dvp', float, dimensions=('nlat', 'nlon','ndepth'))
    nc['dvp'][:,:,:] = MLTomo[:,:,:]


    if imodel == 0:
        nc['dvp'][:,:,:] = np.zeros(np.shape(MLTomo[:,:,:]))
        nc.close()
        continue

    nc.close()



../Runs/SphericalHarmonicsAcousticBall0000  created


/tmp/ipykernel_594233/1365052768.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  TomoSum[:,:] = TomoSum[:,:] + coeff[SlicingDepth][name] * Y_grid


-0.5959282102865157 0.5881323885603876
../Runs/SphericalHarmonicsAcousticBall0001  created


/tmp/ipykernel_594233/1365052768.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  TomoSum[:,:] = TomoSum[:,:] + coeff[SlicingDepth][name] * Y_grid


-0.5346134427793557 0.5645370991737617
../Runs/SphericalHarmonicsAcousticBall0002  created


...

-0.5411625257246284 0.4666712071743219
../Runs/SphericalHarmonicsAcousticBall0018  created


/tmp/ipykernel_594233/1365052768.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  TomoSum[:,:] = TomoSum[:,:] + coeff[SlicingDepth][name] * Y_grid


-0.5796188823073999 0.5166950875338313
../Runs/SphericalHarmonicsAcousticBall0019  created


/tmp/ipykernel_594233/1365052768.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  TomoSum[:,:] = TomoSum[:,:] + coeff[SlicingDepth][name] * Y_grid


-0.5392167118186488 0.4939921096191986


In [7]:
MLtomo_ptb_smooth.min(), MLtomo_ptb_smooth.max()

(-0.04590417695446651, 0.04053691999487109)

In [8]:
MLtomo_ptb.min(), MLtomo_ptb.max()

(-0.09998533711018423, 0.09996325148616125)